In [100]:
from sklearn.datasets import load_wine
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

In [102]:
c1 = SGDClassifier()
c2 = DecisionTreeClassifier(max_depth = 5)
c3 = LogisticRegression(multi_class = "multinomial", max_iter = 10)
c4 = RandomForestClassifier(max_depth = 5)
cs = [c1, c2, c3, c4]

In [103]:
import pandas as pd
import numpy as np

In [104]:
wine = load_wine()

In [105]:
data=pd.DataFrame(data=np.c_[wine['data'],wine['target']],columns=wine['feature_names']+['target'])

In [106]:
data.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0.0


In [107]:
def sample(data, b, n):
    bag=[]
    for i in range(b):
        bag.append(data.sample(n, replace = True))
    return bag

In [119]:
bag = sample(data,4,100)

In [120]:
bag[0] 

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
65,12.37,1.21,2.56,18.1,98.0,2.42,2.65,0.37,2.08,4.60,1.19,2.30,678.0,1.0
12,13.75,1.73,2.41,16.0,89.0,2.60,2.76,0.29,1.81,5.60,1.15,2.90,1320.0,0.0
19,13.64,3.10,2.56,15.2,116.0,2.70,3.03,0.17,1.66,5.10,0.96,3.36,845.0,0.0
20,14.06,1.63,2.28,16.0,126.0,3.00,3.17,0.24,2.10,5.65,1.09,3.71,780.0,0.0
95,12.47,1.52,2.20,19.0,162.0,2.50,2.27,0.32,3.28,2.60,1.16,2.63,937.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13.17,5.19,2.32,22.0,93.0,1.74,0.63,0.61,1.55,7.90,0.60,1.48,725.0,2.0
19,13.64,3.10,2.56,15.2,116.0,2.70,3.03,0.17,1.66,5.10,0.96,3.36,845.0,0.0
76,13.03,0.90,1.71,16.0,86.0,1.95,2.03,0.24,1.46,4.60,1.19,2.48,392.0,1.0
83,13.05,3.86,2.32,22.5,85.0,1.65,1.59,0.61,1.62,4.80,0.84,2.01,515.0,1.0


In [121]:
c_bag = pd.concat([bag[0],bag[1],bag[2],bag[3]])

In [122]:
oob = pd.concat([data,c_bag])
oob = oob[~oob.index.duplicated(keep=False)]
len(oob)

21

In [123]:
def train():
    for i, c in enumerate(cs):
            X_bag = bag[i].drop(columns=['target'])
            y_bag = bag[i]['target']
            c.fit(X_bag, y_bag)
            score = c.score(X_bag, y_bag)
            print(("{} \nscore: {}").format(type(c), score))

In [124]:
train()

<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'> 
score: 0.63
<class 'sklearn.tree._classes.DecisionTreeClassifier'> 
score: 1.0
<class 'sklearn.linear_model._logistic.LogisticRegression'> 
score: 0.71


c:\Users\uzi16\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


<class 'sklearn.ensemble._forest.RandomForestClassifier'> 
score: 1.0


In [125]:
def test():
    X_test = oob.drop(columns=['target'])
    y_test = oob['target']
    for c in cs:
        score = c.score(X_test, y_test)
        print(("{} \nscore: {}").format(type(c),score ))

In [126]:
test()

<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'> 
score: 0.5714285714285714
<class 'sklearn.tree._classes.DecisionTreeClassifier'> 
score: 0.9047619047619048
<class 'sklearn.linear_model._logistic.LogisticRegression'> 
score: 0.7619047619047619
<class 'sklearn.ensemble._forest.RandomForestClassifier'> 
score: 0.9523809523809523


In [129]:
cst = [('SGD', c1), ('DT', c2), ('LR', c3),('RF', c4)]
vch = VotingClassifier(cst, voting = 'hard', verbose = True)

X_train = c_bag.drop(columns=['target'])
y_train = c_bag['target']
X_test = oob.drop(columns=['target'])
y_test = oob['target']
vch.fit(X_train, y_train)

[Voting] ...................... (1 of 4) Processing SGD, total=   0.0s
[Voting] ....................... (2 of 4) Processing DT, total=   0.0s
[Voting] ....................... (3 of 4) Processing LR, total=   0.0s


c:\Users\uzi16\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[Voting] ....................... (4 of 4) Processing RF, total=   0.3s


VotingClassifier(estimators=[('SGD', SGDClassifier()),
                             ('DT', DecisionTreeClassifier(max_depth=5)),
                             ('LR',
                              LogisticRegression(max_iter=10,
                                                 multi_class='multinomial')),
                             ('RF', RandomForestClassifier(max_depth=5))],
                 verbose=True)

In [130]:
vch.score(X_train, y_train), vch.score(X_test, y_test)

(0.9725, 0.9047619047619048)

POPRAWIONE ZADANIE NIŻEJ, NIE CHCIAŁEM USUWAĆ TEGO CO JUŻ WCZEŚNIEJ BYŁO.
Z CIEKAWOŚCI DODAŁEM JESZCZE JEDEN MODEL I JUŻ MI SIĘ NIE CHCE GO USUWAĆ

In [318]:
oob.iloc[[20]]


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
174,13.4,3.91,2.48,23.0,102.0,1.8,0.75,0.43,1.41,7.3,0.7,1.56,750.0,2.0


In [319]:
def superGigaVotingClassifier(models, input):
    votes = []
    for i in models:
        votes.append(i.predict(input)[0])
    counter = Counter(votes)
    vals = max(counter.values())
    return [k for k, v in counter.items() if v == vals]

In [320]:
decision = superGigaVotingClassifier(cs, X_test.iloc[[20]])

In [321]:
if len(decision) > 1:
  print(f"{decision[0]} and {decision[1]}")
else:
  print(f"{decision[0]}")

2.0


In [322]:
decision

[2.0]

In [325]:
index = list(X_test.index)
print(index)
count = 0
for i in index:
    row = X_test.loc[[i]]
    output = superGigaVotingClassifier(cs, row)
    target = y_test[i]
    print(target)
    if(output[0]==target):
        count+=1
    print(f"{output[0]}")

score = count/len(index)
print(score)

[0, 2, 3, 8, 27, 32, 48, 51, 68, 73, 78, 81, 94, 98, 102, 113, 134, 135, 137, 144, 174]
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
2.0
2.0
2.0
2.0
1.0
2.0
0.0
2.0
2.0
0.8571428571428571
